In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from sklearn.utils import shuffle
from tensorflow.keras import optimizers
from sklearn.neighbors import KNeighborsClassifier

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.nasnet import NASNetLarge
from tensorflow.keras import models

import numpy as np

from datetime import datetime



### Загрузка данных

In [2]:
BATCH = 128
image_size = 150
def generator():
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_data = datagen.flow_from_directory('/kaggle/input/intel-image-classification/seg_train/seg_train/',
                                        target_size=(image_size, image_size),
                                        batch_size=BATCH,
                                        class_mode='categorical',
                                        shuffle=True)

    test_data = datagen.flow_from_directory('/kaggle/input/intel-image-classification/seg_test/seg_test/',
                                        target_size=(image_size, image_size),
                                        batch_size=BATCH,
                                        class_mode='categorical',
                                        shuffle=True)

    return train_data, test_data
train_data, test_data = generator()
train_images, train_labels = train_data.next()
test_images, test_labels = test_data.next()

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [3]:
print(len(test_data))
print(test_data[23][0].shape)

print(len(train_data))
print(train_data[109][0].shape)

24
(56, 150, 150, 3)
110
(82, 150, 150, 3)


In [4]:
train_data[0][0].shape

(128, 150, 150, 3)

In [5]:
train_data.class_indices

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

### GPU Info

In [6]:
import subprocess
import pprint

sp = subprocess.Popen(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

out_str = sp.communicate()
out_list = str(out_str[0]).split('\\n')

out_dict = {}

for item in out_list:
    print(item)

b'Tue Dec 24 11:16:11 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU

### Эксперимент 1
Использование структуры глубокой модели, построенной для решения исходной «Задачи А», с целью обучения аналогичной модели для решения «Задачи В»
* Предполагается, что модель, построенная для решения исходной задачи, обучается на данных, подготовленных для
решения целевой задачи
* При этом веса модели инициализируются случайным образом
* Эксперимент реализует перенос знаний для родственных доменов

Используем VGG16 - модель для решения задачи ImageNet.

In [7]:
ACTIVATION='relu'
KERNEL_INIT='he_normal'
vgg_model = Sequential()
vgg_model.add(VGG16(include_top=False, weights=None, input_shape=(image_size, image_size, 3), classes=6))
vgg_model.add(layers.Flatten())
vgg_model.add(layers.Dense(1000, activation=ACTIVATION, input_dim=4*4*512, kernel_initializer=KERNEL_INIT))
vgg_model.add(layers.Dense(6, activation='softmax'))

vgg_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 1000)              8193000   
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 6006      
Total params: 22,913,694
Trainable params: 22,913,694
Non-trainable params: 0
_________________________________________________________________


In [8]:
EPOCHS = 20
vgg_model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
time_start = datetime.now()
history = vgg_model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
time = datetime.now() - time_start
print('Time: ', time)

Train for 110 steps, validate for 24 steps
Epoch 1/20
110/110 [==============================] - 66s 601ms/step - loss: 1394.3259 - accuracy: 0.1816 - val_loss: 1.7916 - val_accuracy: 0.1750
Epoch 2/20
110/110 [==============================] - 53s 479ms/step - loss: 1.8321 - accuracy: 0.1719 - val_loss: 1.7909 - val_accuracy: 0.1790
Epoch 3/20
110/110 [==============================] - 53s 480ms/step - loss: 1.8653 - accuracy: 0.1768 - val_loss: 1.7853 - val_accuracy: 0.1750
Epoch 4/20
110/110 [==============================] - 53s 481ms/step - loss: 3.7933 - accuracy: 0.1905 - val_loss: 1.7903 - val_accuracy: 0.1750
Epoch 5/20
110/110 [==============================] - 53s 479ms/step - loss: 2.1756 - accuracy: 0.1939 - val_loss: 1.6877 - val_accuracy: 0.2503
Epoch 6/20
110/110 [==============================] - 53s 478ms/step - loss: 2.1238 - accuracy: 0.2173 - val_loss: 1.7889 - val_accuracy: 0.1670
Epoch 7/20
110/110 [==============================] - 53s 478ms/step - loss: 1.8309 

### Эксперимент 2
Использование модели, построенной для решения исходной «Задачи А», в качестве фиксированного
метода извлечения признаков при построении модели, решающей «Задачу В»
* Идея данного подхода состоит в том, чтобы удалить из глубокой модели классификатор (последние полностью
связанные слои) и рассматривать начальную часть сети как метод выделения признаков
* Взамен старого классификатора можно поместить новый классификатор (например, другой набор полностью
связанных слоев или машину опорных векторов) и обучить его на признаках, построенных с использованием начальной
части сети
* Эксперимент реализует перенос признакового описания

Возьмем за основу сверточную нейронную сеть VGG16, обученную на наборе ImageNet (1,4 миллиона изобр, классифицированных на 1000 классов).

Пропустим набор данных intel-image через предварительно обученную сверточную основу VGG16, запишем результат.

Используем результат как входные данные для отдельного полносвязного классификатора.

In [9]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3), classes=1000)
vgg_model.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

In [10]:
def extract_features(sample_count, vgg_model, generat):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count, 6))
    i=0
    for inputs_batch, labels_batch in generat:
        #выделяем признаки из изображений
        features_batch = vgg_model.predict(inputs_batch)
        features[i * BATCH : (i + 1) * BATCH] = features_batch
        labels[i * BATCH : (i + 1) * BATCH] = labels_batch
        i = i + 1
        if (i + 1) * BATCH >= sample_count:
            return features, labels
    return features, labels

In [11]:
time_start = datetime.now()
train_sample_count = 14034
test_sample_count = 3000
generat = generator()
#выделяем признаки
train_features, train_y = extract_features(train_sample_count, vgg_model, generat[0])
test_features, test_y = extract_features(test_sample_count, vgg_model, generat[1])

time = datetime.now() - time_start
print('Time: ', time)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Time:  0:00:56.887767


In [12]:
print(train_features.shape)
print(train_y.shape)

print(test_features.shape) 
print(test_y.shape)

(14034, 4, 4, 512)
(14034, 6)
(3000, 4, 4, 512)
(3000, 6)


In [13]:
train_features = np.reshape(train_features, (train_sample_count, 4*4*512))
test_features = np.reshape(test_features, (test_sample_count, 4*4*512))

In [14]:
print(train_features.shape)
print(train_y.shape)

(14034, 8192)
(14034, 6)


In [15]:
#передадим полученные признаки на вход полносвязному классификатору
EPOCHS = 20
ACTIVATION='relu'
KERNEL_INIT='he_normal'

model = Sequential()
model.add(layers.Dense(1000, activation=ACTIVATION, input_dim=4*4*512, kernel_initializer=KERNEL_INIT))

model.add(layers.Dense(6, activation='softmax'))
model.summary()

model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
time_start = datetime.now()
history = model.fit(train_features, train_y, epochs=EPOCHS, batch_size = 128, shuffle=True, validation_data=(test_features, test_y))
time = datetime.now() - time_start
print('Time: ', time)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1000)              8193000   
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 6006      
Total params: 8,199,006
Trainable params: 8,199,006
Non-trainable params: 0
_________________________________________________________________
Train on 14034 samples, validate on 3000 samples
Epoch 1/20
14034/14034 [==============================] - 2s 118us/sample - loss: 1.2050 - accuracy: 0.7291 - val_loss: 0.4148 - val_accuracy: 0.8413
Epoch 2/20
14034/14034 [==============================] - 1s 76us/sample - loss: 0.4366 - accuracy: 0.8395 - val_loss: 0.6488 - val_accuracy: 0.7817
Epoch 3/20
14034/14034 [==============================] - 1s 78us/sample - loss: 0.3457 - accuracy: 0.8712 - val_loss: 0.3623 - val_accuracy: 0.8587
Epoch 4/2

### Эксперимент 3
Тонкая настройка параметров модели, построенной для решения исходной «Задачи А», с целью решения
«Задачи В»
* Последние слои глубокой модели, соответствующие классификатору, который решает «Задачу А», заменяются
новым классификатором (например, набором полностью связанных слоев с другим количеством выходов)
* Полученная модель обучается как единая система 
* Эксперимент реализует перенос обучения на основе
экземпляров

In [16]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3), classes=1000)
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [17]:
EPOCHS = 20
ACTIVATION='relu'
KERNEL_INIT='he_normal'

model = Sequential()
model.add(vgg_model)
vgg_model.trainable = False
model.add(layers.Flatten())
model.add(layers.Dense(1000, activation=ACTIVATION, input_dim=4*4*512, kernel_initializer=KERNEL_INIT))

model.add(layers.Dense(6, activation='softmax'))
model.summary()

model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
time_start = datetime.now()
history =  model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
time = datetime.now() - time_start
print('Time: ', time)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              8193000   
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 6006      
Total params: 22,913,694
Trainable params: 8,199,006
Non-trainable params: 14,714,688
_________________________________________________________________
Train for 110 steps, validate for 24 steps
Epoch 1/20
110/110 [==============================] - 35s 322ms/step - loss: 1.2433 - accuracy: 0.7334 - val_loss: 0.3783 - val_accuracy: 0.8577
Epoch 2/20
110/110 [========================